<a href="https://colab.research.google.com/github/CAVASOL/aiffel_quest/blob/main/Reference/30vnfoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3-1. 프로젝트: 베트남 음식 분류하기

**Index**

Set up  
Remove broken files  
Check data for training  
Dataloader  
Create a model  
Custom Trainer  
Training  
Test model  

### Set up

In [ ]:
import os
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

### Remove broken files

In [ ]:
# training set과 test set의 모든 이미지 파일에 대해서,
# jpg image header가 포함되지 않은 (jpg의 파일 구조에 어긋나는) 파일들을 삭제해줍니다.

data_path = '/aiffel/aiffel/model-fit/data/30vnfoods/'
train_path = data_path + 'Train/'
test_path = data_path + 'Test/'

for path in [train_path, test_path]:
    classes = os.listdir(path)

    for food in classes:
        food_path = os.path.join(path, food)
        images = os.listdir(food_path)

        for image in images:
            with open(os.path.join(food_path, image), 'rb') as f:
                bytes = f.read()
            if bytes[:3] != b'\xff\xd8\xff':
                print(os.path.join(food_path, image))
                os.remove(os.path.join(food_path, image))

### Check data for training

In [ ]:
classes = os.listdir(train_path)
train_length = 0

for food in classes:
    food_path = os.path.join(train_path, food)
    images = os.listdir(food_path)

    train_length += len(images)

print('training data의 개수: '+str(train_length))

training data의 개수: 9775


### Dataloader

In [ ]:
# 문제 1: dataloader 구현하기

def process_path(file_path, class_names, img_shape=(224, 224)):
    '''
    file_path로 부터 class label을 만들고, 이미지를 읽는 함수
    이미지 크기를 (224, 224)로 맞춰주세요.
    '''
    label = tf.strings.split(file_path, os.path.sep)
    label = label[-2] == class_names

    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, img_shape)

    return img, label

def prepare_for_training(ds, batch_size=32, cache=True, shuffle_buffer_size=1000):
    '''
    TensorFlow Data API를 이용해 data batch를 만드는 함수
    '''
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat()
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

def load_data(data_path, batch_size=32):
    '''
    데이터를 만들기 위해 필요한 함수들을 호출하고 데이터를 리턴해주는 함수
    TensorFlow Dataset 객체를 생성하고 process_path 함수로 이미지와 라벨을 묶은 다음,
    prepare_for_training 함수로 batch가 적용된 Dataset 객체를 만들어주세요.
    '''
    class_names = [cls for cls in os.listdir(data_path) if cls != '.DS_Store']
    print(class_names)
    data_path = pathlib.Path(data_path)

    list_ds = tf.data.Dataset.list_files(str(data_path/'*/*'))
    labeled_ds = list_ds.map(lambda x: process_path(x, class_names, img_shape=(224, 224)))
    ds = prepare_for_training(labeled_ds, batch_size=batch_size)

    return ds

### Create a model

In [ ]:
# 문제 2: 모델 구현하기

from tensorflow.keras.applications import EfficientNetB0

class Model(tf.keras.Model):
    '''
    EfficientNetB0을 백본으로 사용하는 모델을 구성합니다.
    Classification 문제로 접근할 것이기 때문에 맨 마지막 Dense 레이어에
    우리가 원하는 클래스 개수만큼을 지정해주어야 합니다.
    '''
    def __init__(self, num_classes=10, freeze=False):
        super(Model, self).__init__()
        self.base_model = EfficientNetB0(include_top=False, weights='imagenet')
        if freeze:
            self.base_model.trainable = False
        self.top = tf.keras.Sequential([tf.keras.layers.GlobalAveragePooling2D(name="avg_pool"),
                                       tf.keras.layers.BatchNormalization(),
                                       tf.keras.layers.Dropout(0.5, name="top_dropout")])
        self.classifier = tf.keras.layers.Dense(num_classes, activation="softmax", name="pred")
    def call(self, inputs, training=True):
        x = self.base_model(inputs)
        x = self.top(x)
        x = self.classifier(x)
        return x

if __name__ == '__main__':
    model = Model(num_classes=5, freeze=True)
    model.build(input_shape=(None, 224, 224, 3))
    print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, None, None, 1280)  4049571   
_________________________________________________________________
sequential (Sequential)      (None, 1280)              5120      
_________________________________________________________________
pred (Dense)                 multiple                  6405      
Total params: 4,061,096
Trainable params: 8,965
Non-trainable params: 4,052,131
_________________________________________________________________
None


### Custom Trainer

In [ ]:
# 문제 3: custom trainer 구현하기

class Trainer:
    def __init__(self, model, epochs, batch, ds_length, loss_fn, optimizer):
        self.model = model
        self.epochs = epochs
        self.batch = batch
        self.loss_fn = loss_fn
        self.optimizer = optimizer
    def train(self, train_dataset, train_metric):
        for epoch in range(self.epochs):
            print("\nStart of epoch %d" % (epoch+1,))
            for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
                with tf.GradientTape() as tape:
                    logits = model(x_batch_train, training=True)
                    loss_value = self.loss_fn(y_batch_train, logits)
                grads = tape.gradient(loss_value, model.trainable_weights)
                self.optimizer.apply_gradients(zip(grads, model.trainable_weights))
                # train metric 업데이트
                train_metric.update_state(y_batch_train, logits)
                # 5 배치마다 로깅
                if step % 5 == 0:
                    print(
                        "Training loss (for one batch) at step %d: %.4f"
                        % (step, float(loss_value))
                    )
                    print("Seen so far: %d samples" % ((step + 1) * self.batch))
                    print(train_metric.result().numpy())
            # 마지막 epoch 학습이 끝나면 train 결과를 보여줌
            train_acc = train_acc_metric.result()
            print("Training acc over epoch: %.4f" % (float(train_acc),))

### Training

In [ ]:
# 모델 학습 코드

train_path = "/aiffel/aiffel/model-fit/data/30vnfoods/Train"

epoch = 5
batch = 32

model = Model(num_classes=10)
dataset = load_data(data_path=train_path, batch_size=batch)
loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
trainer = Trainer(model=model,
                epochs=epoch,
                batch=batch,
                ds_length=train_length,
                loss_fn=loss_function,
                optimizer=optimizer)

trainer.train(train_dataset=dataset,
            train_metric=train_acc_metric)

['Bun rieu', 'Banh mi', 'Banh xeo', 'Chao long', 'Pho', 'Banh khot', 'Bun bo Hue', 'Banh cuon', 'Com tam', 'Bun dau mam tom']

Start of epoch 1
Training loss (for one batch) at step 0: 3.5719
Seen so far: 32 samples
0.125
Training loss (for one batch) at step 5: 2.8947
Seen so far: 192 samples
0.17708333
Training loss (for one batch) at step 10: 2.6582
Seen so far: 352 samples
0.21306819
Training loss (for one batch) at step 15: 2.4352
Seen so far: 512 samples
0.26171875
Training loss (for one batch) at step 20: 1.0362
Seen so far: 672 samples
0.32291666
Training loss (for one batch) at step 25: 1.2399
Seen so far: 832 samples
0.36177886
Training loss (for one batch) at step 30: 1.1131
Seen so far: 992 samples
0.40221775
Training loss (for one batch) at step 35: 1.4706
Seen so far: 1152 samples
0.43489584
Training loss (for one batch) at step 40: 1.0896
Seen so far: 1312 samples
0.46417683
Training loss (for one batch) at step 45: 1.0042
Seen so far: 1472 samples
0.4816576
Training los

ResourceExhaustedError: OOM when allocating tensor with shape[1280] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:FusedBatchNormV3]

### Test model

In [ ]:
# 모델 테스트 코드

test_ds = load_data(data_path=test_path)

for step_train, (x_batch_train, y_batch_train) in enumerate(test_ds.take(10)):
    prediction = model(x_batch_train)
    print("{}/{}".format(np.array(tf.equal(tf.argmax(y_batch_train, axis=1), tf.argmax(prediction, axis=1))).sum(), tf.argmax(y_batch_train, axis=1).shape[0]))

['Bun rieu', 'Banh mi', 'Banh xeo', 'Chao long', 'Pho', 'Banh khot', 'Bun bo Hue', 'Banh cuon', 'Com tam', 'Bun dau mam tom']


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run Mul: Dst tensor is not initialized. [Op:Mul]